In [ ]:
# %pip install pyspark

from pyspark import SparkContext, SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col, monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, StringType

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk:1.12.708,org.apache.hadoop:hadoop-aws:3.3.4 pyspark-shell'

conf = SparkConf().set('spark.executor.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true'). \
 set('spark.driver.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true'). \
 setAppName('pyspark_aws').setMaster('local[*]')
            
sc=SparkContext(conf=conf)
sc.setSystemProperty('com.amazonaws.services.s3.enableV4', 'true')

accessKeyId='**************'
secretAccessKey='****************************'

hadoopConf = sc._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', accessKeyId)
hadoopConf.set('fs.s3a.secret.key', secretAccessKey)
hadoopConf.set('fs.s3a.endpoint', 's3-us-east-1.amazonaws.com')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

spark=SparkSession(sc)

In [ ]:
df_schema=StructType([StructField('Scorecard', StringType(), True), 
                        StructField('Team 1', StringType(), True), 
                        StructField('Team 2', StringType(), True), 
                        StructField('Winner', StringType(), True),
                        StructField('Margin', StringType(), True), 
                        StructField('Ground', StringType(), True),
                        StructField('Match Date', StringType(), True)
                       ])

df = spark.read.csv('../files/cricketMatchesDataset.csv', header=True)
df.count()

In [ ]:
india_df = df.select(col('Team 1'), 
                     col('Team 2'), 
                     col('Winner'), 
                     col('Ground'), 
                     col('Match Date')
                    ).where((df["Team 1"].contains("India")) | (df["Team 2"].contains("India"))
                           ).withColumn('id', monotonically_increasing_id()).cache()
india_df.count()

In [ ]:
india_df.write.format('csv').option('header','true').save('path/to/bucket/filename.csv',mode='overwrite')